In [1]:
from mario_replays.load_data import load_replay_data
import os
import pandas as pd

sourcedata = '../../mario_learning.behavior/sourcedata/'

In [2]:
clips_df = load_replay_data(os.path.join(sourcedata, 'scene_clips'), type='metadata')
clips_df

,Subject,World,Level,Duration,Cleared,ScoreGained,X_Traveled,Lives_lost,Hits_taken,Enemies_killed,...,StartFrame,EndFrame,TotalFrames,Bk2Filepath,GameName,LevelFullName,IndexInRun,IndexGlobal,IndexLevel,Phase
0,01,2,1,4.483333,False,50,82,1,1,0,...,1814,2082,5702,sub-01/ses-001/gamelogs/sub-01_ses-001_task-ma...,SuperMarioBros-Nes,w2l1,5,38,5,discovery
1,01,1,2,2.450000,True,0,174,0,0,0,...,1127,1273,3098,sub-01/ses-001/gamelogs/sub-01_ses-001_task-ma...,SuperMarioBros-Nes,w1l2,5,16,5,discovery
2,01,1,2,3.083333,True,10,250,0,0,0,...,840,1024,3090,sub-01/ses-001/gamelogs/sub-01_ses-001_task-ma...,SuperMarioBros-Nes,w1l2,1,12,1,discovery
3,01,1,3,3.566667,True,0,159,0,0,0,...,1164,1377,2597,sub-01/ses-001/gamelogs/sub-01_ses-001_task-ma...,SuperMarioBros-Nes,w1l3,8,29,8,discovery
4,01,1,3,6.066667,False,0,156,1,1,0,...,2164,2527,2528,sub-01/ses-001/gamelogs/sub-01_ses-001_task-ma...,SuperMarioBros-Nes,w1l3,1,22,1,discovery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56136,06,8,2,0.916667,True,0,135,0,0,0,...,3767,3821,6729,sub-06/ses-025/gamelogs/sub-06_ses-025_task-ma...,SuperMarioBros-Nes,w8l2,2,608,27,practice
56137,06,5,2,14.316667,True,100,190,0,0,0,...,2751,3609,8074,sub-06/ses-025/gamelogs/sub-06_ses-025_task-ma...,SuperMarioBros-Nes,w5l2,1,585,30,practice
56138,06,8,2,2.033333,True,0,162,0,0,0,...,721,842,6729,sub-06/ses-025/gamelogs/sub-06_ses-025_task-ma...,SuperMarioBros-Nes,w8l2,2,608,27,practice
56139,06,5,3,2.416667,True,0,129,0,0,0,...,133,277,5862,sub-06/ses-025/gamelogs/sub-06_ses-025_task-ma...,SuperMarioBros-Nes,w5l3,2,591,26,practice


In [3]:
repetitions_df = load_replay_data(os.path.join(sourcedata, 'replays'), type='metadata')
repetitions_df

,Subject,World,Level,Duration,Cleared,ScoreGained,X_Traveled,Lives_lost,Hits_taken,Enemies_killed,Powerups_collected,Bricks_destroyed,CoinsGained,IndexInRun,Run,IndexGlobal,IndexLevel,Phase,LevelFullName
0,01,1,1,40.616667,True,2105,3116,0,0,5,0,0,2,3,run-01,3,3,discovery,w1l1
1,01,1,3,58.216667,False,1985,2371,2,2,2,0,0,7,5,run-03,26,5,discovery,w1l3
2,01,2,1,58.950000,False,220,2278,3,3,6,0,0,0,6,run-04,39,6,discovery,w2l1
3,01,1,3,58.850000,False,1920,2376,2,2,2,0,0,5,2,run-03,23,2,discovery,w1l3
4,01,1,1,80.516667,False,1900,3127,2,2,4,0,0,2,0,run-01,0,0,discovery,w1l1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3369,06,7,1,180.750000,False,2915,2824,2,2,10,1,2,19,1,run-03,596,26,practice,w7l1
3370,06,3,3,75.666667,True,1935,2360,0,0,11,1,0,21,3,run-02,592,27,practice,w3l3
3371,06,2,3,46.366667,True,1820,3536,0,0,0,0,0,10,0,run-03,595,28,practice,w2l3
3372,06,1,2,142.700000,True,2150,2817,-1,0,15,2,18,69,4,run-01,588,23,practice,w1l2


In [4]:
repetitions_variables = load_replay_data(os.path.join(sourcedata, 'replays'), type='variables')

In [5]:
clips_variables = load_replay_data(os.path.join(sourcedata, 'scene_clips'), type='variables')

In [2]:
from PIL import Image

sourcedata = os.path.join('/media/hyruuk/Commons/DATA/mario_data/mario.scenes/sourcedata/')

# load images
level_backgrounds_folder = 'level_backgrounds'
scene_backgrounds_folder = 'scene_backgrounds'

In [4]:
from mario_scenes.load_data import load_background_images
from mario_scenes.load_data import load_scenes_info

level_backgrounds = load_background_images(level='level')
scene_backgrounds = load_background_images(level='scene')
scenes_info = load_scenes_info()

In [6]:
scene_backgrounds[0].size

(200, 224)

In [5]:
scenes_info

,World,Level,Scene,x Hi (entry),x Lo (entry),x Hi (exit),x Lo (exit),Entry point,Exit point,Layout,...,Empty stair valley,Enemy stair valley,Gap stair valley,Reward,Moving platform,Flagpole,Beginning,Bonus zone,Waterworld,Checkpoint
0,1.0,1.0,0.0,0.0,40.0,0.0,240.0,40.0,240.0,144.0,...,0,0,0,0,0,0,1,0,0,0
1,1.0,1.0,1.0,0.0,240.0,1.0,200.0,240.0,456.0,144.0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1.0,2.0,1.0,200.0,2.0,105.0,456.0,617.0,144.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1.0,3.0,2.0,105.0,2.0,232.0,617.0,744.0,144.0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,1.0,4.0,2.0,232.0,3.0,152.0,744.0,920.0,144.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,8.0,3.0,8.0,8.0,150.0,9.0,100.0,2198.0,2404.0,208.0,...,0,0,0,0,0,0,0,0,0,0
310,8.0,3.0,9.0,9.0,100.0,10.0,140.0,2404.0,2700.0,208.0,...,0,0,0,0,0,0,0,0,0,0
311,8.0,3.0,10.0,10.0,140.0,12.0,45.0,2700.0,3117.0,208.0,...,0,0,0,0,0,0,0,0,0,0
312,8.0,3.0,11.0,12.0,45.0,13.0,103.0,3117.0,3431.0,208.0,...,0,0,0,0,0,1,0,0,0,0
